In [1]:
import json
from pathlib import Path

file_path = Path(r"C:/SKN_3rd/data/youth_policies_processed.json"
)

# 파일 읽기
with file_path.open("r", encoding="utf-8") as f:
    policies = json.load(f)

In [2]:
from langchain_core.documents import Document

documents = []

for p in policies:
    # 텍스트(임베딩)에 포함될 내용
    content = f"""
    정책명: {p.get('정책명', '')}
    설명: {p.get('정책설명', '')}
    지원내용: {p.get('지원내용', '')}
    신청방법: {p.get('신청방법', '')}
    제출서류: {p.get('제출서류', '')}
    키워드: {p.get('정책키워드', '')}
    """

    # 메타데이터 (검색 필터용)
    metadata = {
        "policy_name": p.get("정책명"),
        "big_category": p.get("대분류"),
        "mid_category": p.get("중분류"),
        "region_sido": p.get("region_sido"),
        "region_sigungu": p.get("region_sigungu"),
        "region_level": p.get("region_level"),
        "age_min": int(p.get("지원최소연령") or 0),
        "age_max": int(p.get("지원최대연령") or 100),
        "org": p.get("주관기관명"),
        "register_org": p.get("등록기관명"),
        "income_condition": p.get("소득조건"),
        "major_condition": p.get("전공요건"),
        "employment_condition": p.get("취업상태"),
        "degree_condition": p.get("학력요건"),
        "extra_condition": p.get("추가자격조건"),
        "apply_period": p.get("신청기간"),
        "apply_type": p.get("신청기간구분"),
        "policy_type": p.get("정책제공방법"),
        "support_min": p.get("최소지원금액"),
        "support_max": p.get("최대지원금액"),
        "policy_id": p.get("정책명")  # 필요시 고유 ID 추가
    }

    documents.append(Document(page_content=content.strip(), metadata=metadata))

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

persist_dir = "C:/SKN_3rd/chroma_db/youth_policy_advanced"

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Chroma.from_documents(
    documents=documents,
    collection_name="youth_policy_advanced",
    embedding=embedding_model,
    persist_directory=persist_dir
)

print("벡터 저장 완")  # 자동 저장됨

벡터 저장 완


# Retreiver
- self-query
- ensenmble
- multi-query

```txt
사용자 질문
   ↓
Self-Query (자연어 → 필터 + 키워드)
   ↓
필터 기반 Retriever (정확성 ↑)
   ↓
BM25 + VectorRetriever Ensemble (누락 ↓)
   ↓
MultiQueryRetriever (검색 확장 ↑)
   ↓
최종 문서


In [5]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

persist_dir = "C:/SKN_3rd/chroma_db/youth_policy_advanced"

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Chroma(
    collection_name="youth_policy_advanced",
    embedding_function=embedding_model,
    persist_directory=persist_dir
)

print("벡터 로드 완")

벡터 로드 완


In [11]:
from langchain_chroma import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain_openai import OpenAIEmbeddings
from langchain_classic.retrievers import EnsembleRetriever